# Designing Analysis Classes

We will now learn how to create your own Analysis class by going through the implementation of the `BasicBrainAnalysis` class from the "Applying Analysis Classes" notebook.

There are three key components of an Analysis class:

* Data specification
* Parameter specification
* Pipeline constructor methods

The data specification defines all inputs, outputs and intermediate derivatives. The parameter specification specifies the free (meta) parameters that can be used to customise the analysis. Pipeline constructor methods are just regular methods of the class that return a `Pipeline` that generates one or more derivatives.

## Base and Meta-classes and Inheritance

Every Analysis class should (at least indirectly) inherit from the `arcana.Analysis` base class. This contains methods to perform a lot of the "magic" that Arcana does. 

Arcana is designed to utilise inheritance of Analysis classes, but since Analysis classes specify a number of class attributes (i.e. for the data and parameter specifications) all Analysis classes need to be constructed by a special "meta-class", `arcana.AnalysisMetaClass`. However, you don't need to understand what is going on behind the scenes (or what a meta-class is even), just simply define your class like this


In [5]:
from arcana import Analysis, AnalysisMetaClass

class MyBasicBrainAnalysis(Analysis, metaclass=AnalysisMetaClass):
    
    # Stuff goes here
    pass

## Data specification

The data specification is the place to start designing an Analysis class. As the name suggests, it specifies all inputs, outputs and intermediate derivatives of the class via a list of "data-spec" objects:

* `FilesetSpec` (intermediate file-set derivatives)
* `InputFilesetSpec`
* `OutputFilesetSpec`
* `FieldSpec` (intermediate field derivatives)
* `InputFieldSpec`
* `OutputFieldSpec`

Instead of setting the data specification directly, data-spec objects are appended to the specifications of base classes (by the meta-class) by defining the `add_data_specs` class attribute. This enables the data specifications of base classes to be altered and overwritten.

Each data-spec object is given a name (i.e. the one that appears on the class "menu") and assigned a file-format (filesets) or data type (fields). The key difference between input and output (and intermediate) data-specs is that output data-specs refer to the "pipeline constructor method" that constructs a pipeline to generate them. Intermediate specs are equivalent to output specs in every respect except they appear don't appear on the class menu by default.

Typically, the only thing you will ever need to do with a data-spec is initialise it and append it to the `add_data_specs` list, and the best place to see the available initialisation options is the doc strings, i.e.

In [6]:
from arcana import InputFilesetSpec, InputFieldSpec, FilesetSpec, FieldSpec
print('InputFilesetSpec:\n', InputFilesetSpec.__doc__)
print('InputFieldSpec:\n', InputFieldSpec.__doc__)
print('FilesetSpec:\n', FilesetSpec.__doc__)
print('FieldSpec:\n', FieldSpec.__doc__)

InputFilesetSpec:
 
    A specification for an "acquired" fileset (e.g from the scanner or
    standard atlas)

    Parameters
    ----------
    name : str
        The name of the fileset
    valid_formats : FileFormat | list[FileFormat]
        The acceptable file formats for input filesets to match this spec
    frequency : str
        One of 'per_session', 'per_subject', 'per_visit' or 'per_dataset',
        specifying whether the fileset is present for each session, subject,
        visit or dataset.
    desc : str
        Description of what the field represents
    optional : bool
        Whether the specification is optional or not. Only valid for
        "acquired" fileset specs.
    default : FilesetSlice | object
        The default value to be passed as an input to this spec if none are
        provided. Can either be an explicit FilesetSlice or any object
        with a 'slice' property that will return a default slice.
        This object should also implement a 'bind(sel

Key parameters to note are:

#### file_format/datatype

The format/data-type that the item will be converted to when it is stored in the dataset

#### frequency

Where the "data-items" sit in the dataset tree, i.e. whether there is one for every session, subject, visit or the whole dataset. Valid values for `frequency` are

* 'per_session'
* 'per_subject'
* 'per_visit'
* 'per_dataset'

#### pipeline_getter

The name of the method in the class that constructs the pipeline to generate the derivatives

### Example

In the Basic-Brain Analysis class we define three output, one input and one intermediate fileset specs as such

In [7]:
from arcana import InputFilesetSpec, FilesetSpec, OutputFilesetSpec
from banana.file_format import nifti_gz_format

class MyBasicBrainAnalysis(Analysis, metaclass=AnalysisMetaClass):

    add_data_specs = [
        InputFilesetSpec('magnitude', nifti_gz_format,
                         desc="A magnitude image (e.g. T1w, T2w, etc..)"),
        OutputFilesetSpec('brain', nifti_gz_format,
                          'brain_extraction_pipeline',
                          desc="Skull-stripped magnitude image"),
        FilesetSpec('brain_mask', nifti_gz_format,
                    'brain_extraction_pipeline',
                    desc="Brain mask used for skull-stripping"),
        OutputFilesetSpec('smooth', nifti_gz_format, 'smooth_mask_pipeline',
                          desc="Smoothed magnitude image"),
        OutputFilesetSpec('smooth_masked', nifti_gz_format,
                          'smooth_mask_pipeline',
                          desc="Smoothed and masked magnitude image")]
    
    def brain_extraction_pipeline(self, **name_maps):
        "We'll define this later"
    
    def smooth_mask_pipeline(self, **name_maps):
        "We'll define this later"

print(MyBasicBrainAnalysis.static_menu(full=True))


__main__.MyBasicBrainAnalysis Menu 
-----------------------------------

Inputs:
    magnitude : nifti_gz
        A magnitude image (e.g. T1w, T2w, etc..)

Intermediate:
    brain_mask : nifti_gz
        Brain mask used for skull-stripping

Outputs:
    brain : nifti_gz
        Skull-stripped magnitude image
    smooth : nifti_gz
        Smoothed magnitude image
    smooth_masked : nifti_gz
        Smoothed and masked magnitude image

Parameters:


**Note** how the 'pipeline_getter' parameters of the spec objects reference the name of a "pipeline constructor" method defined in the class. The matches between these names are checked by the metaclass so if we don't define them Arcana will throw an error. 

Since `MyBasicBrainAnalysis` inherits directly from `Analysis` the only specs in the data specification are those in `add_data_specs`. However, if we would like to extend `MyBasicBrainAnalysis` to make a new class `MyExtendedBasicBrainAnalysis` we can add to and override the specs from `MyBasicBrainAnalysis`.

In [8]:
from banana.file_format import mrtrix_image_format

class MyExtendedBasicBrainAnalysis(MyBasicBrainAnalysis, metaclass=AnalysisMetaClass):

    add_data_specs = [
        OutputFilesetSpec('smooth', mrtrix_image_format, 'smooth_mask_pipeline',
                          desc="Smoothed magnitude image in Mrtrix format"),
        OutputFilesetSpec('thresholded', nifti_gz_format,
                          'threshold_pipeline',
                          desc="Thresholded smoothed magnitude image")]
    
    def threshold_pipeline(self, **name_maps):
        "We'll define this later"
        
print(MyExtendedBasicBrainAnalysis.static_menu(full=True))


__main__.MyExtendedBasicBrainAnalysis Menu 
-------------------------------------------

Inputs:
    magnitude : nifti_gz
        A magnitude image (e.g. T1w, T2w, etc..)

Intermediate:
    brain_mask : nifti_gz
        Brain mask used for skull-stripping

Outputs:
    brain : nifti_gz
        Skull-stripped magnitude image
    smooth : mrtrix_image
        Smoothed magnitude image in Mrtrix format
    smooth_masked : nifti_gz
        Smoothed and masked magnitude image
    thresholded : nifti_gz
        Thresholded smoothed magnitude image

Parameters:


As you can see, there is now a `thresholded` output, and the `smooth` image is now of `mrtrix_image` format instead of `nifti_gz`. 

**Note**: we can get away with changing the format of the `smooth` from zipped NiFTI to MRtrix because if the output format of the pipeline doesn't match the format of the specification it will be automatically converted before it is stored in the dataset (as long as a converter exists).

## Parameter Specification

The parameter specification works very much like the data specification but for parameters. "Param-specs" can be either of class `ParamSpec` or `SwitchSpec` type.

`SwitchSpec`s are used to qualitatively change the analysis performed, e.g. using FSL for non-linear registration to a template (i.e. FNIRT) instead of ANTs. `ParamSpec`s are used to quantitatively change the analysis, e.g. change the required threshold value.

In [9]:
from arcana import ParamSpec, SwitchSpec

print('ParamSpec:\n', ParamSpec.__doc__)
print('SwitchSpec:\n', SwitchSpec.__doc__)

ParamSpec:
 
    Specifies a parameter that can be passed to the analysis

    Parameters
    ----------
    name : str
        Name of the parameter
    default : float | int | str | list | tuple
        Default value of the parameter
    choices : List(float | int | str | list | tuple) | None
        Restrict valid inputs to the following choices
    desc : str
        A description of the parameter
    dtype : type | None
        The datatype of the parameter. If none will be determined from
        default value
    
SwitchSpec:
 
    Specifies a special parameter that switches between different
    methods and/or pipeline input/outputs. Typically used to select
    between comparable methods (e.g. FSL or ANTs registration) but can
    also be used to specify whether certain methods are applied, and by
    extension some auxiliary outputs are generated

    Parameters
    ----------
    name : str
        Name of the parameter
    default : str
        Default option for the switch

As with the data specification, instead of setting the parameter specification directly it is added to the class via `add_param_specs` to allow manipulation by subclasses.

Returning to the `BasicBrainAnalysis` example we add in the FWHM parameter used in the smoothing pipeline.

In [10]:
class BasicBrainAnalysis(Analysis, metaclass=AnalysisMetaClass):
    """
    A baisc example that demonstrates how Analysis classes work.
    """

    add_data_specs = [
        InputFilesetSpec('magnitude', nifti_gz_format,
                         desc="A magnitude image (e.g. T1w, T2w, etc..)"),
        OutputFilesetSpec('brain', nifti_gz_format,
                          'brain_extraction_pipeline',
                          desc="Skull-stripped magnitude image"),
        FilesetSpec('brain_mask', nifti_gz_format,
                    'brain_extraction_pipeline',
                    desc="Brain mask used for skull-stripping"),
        OutputFilesetSpec('smooth', nifti_gz_format, 'smooth_mask_pipeline',
                          desc="Smoothed magnitude image"),
        OutputFilesetSpec('smooth_masked', nifti_gz_format,
                          'smooth_mask_pipeline',
                          desc="Smoothed and masked magnitude image")]

    add_param_specs = [
        ParamSpec('smoothing_fwhm', 4.0,
                  desc=("The full-width-half-maxium radius of the smoothing "
                        "kernel"))]

    def brain_extraction_pipeline(self, **name_maps):
        "We'll define this later"
    
    def smooth_mask_pipeline(self, **name_maps):
        "We'll define this later"

print(BasicBrainAnalysis.static_menu())


__main__.BasicBrainAnalysis Menu 
---------------------------------

Inputs:
    magnitude : nifti_gz
        A magnitude image (e.g. T1w, T2w, etc..)

Outputs:
    brain : nifti_gz
        Skull-stripped magnitude image
    smooth : nifti_gz
        Smoothed magnitude image
    smooth_masked : nifti_gz
        Smoothed and masked magnitude image

Parameters:
    smoothing_fwhm : float (4.0)
        The full-width-half-maxium radius of the smoothing kernel


## Pipeline Constructor Methods

Pipeline constructor methods are where the action happens in Analysis classes. They return `Pipeline` objects (which are just thin wrappers around `nipype.Workflows`) that link the data specification together by taking one or more data-specs as inputs and generating one or more as outputs.

### Initialising a pipeline

The basic form of a pipeline constructor method is as follows

In [11]:
from banana.citation import fsl_cite

def smooth_mask_pipeline(self, **name_maps):

    pipeline = self.new_pipeline(
        'smooth_mask',
        desc="Smooths and masks a brain image",
        name_maps=name_maps,
        citations=[fsl_cite])

    return pipeline

where the `new_pipeline` creates the `Pipeline` object, which is returned at the end of the method. Looking at the doc string of the `new_pipeline` we can see the parameters that you need/can pass to it

In [12]:
print(Analysis.new_pipeline.__doc__)


        Creates a Pipeline object, passing the analysis (self) as the first
        argument

        Parameters
        ----------
        name : str
            The name of the pipeline
        desc : str
            A description of what the pipeline does
        ciations : List[Citation]
            List of scientific papers that describe the workflow and should be
            cited in publications that use it
        name_maps : dict
            A dictionary containing several modifying keyword arguments that
            manipulate way the pipeline is constructed (e.g. map inputs and
            outputs to new entries in the data specification table). Typically
            names of inputs, outputs and the pipeline itself. Intended to allow
            secondary pipeline constructors to call a constructor, and return a
            modified version of the pipeline it returns.

            It should be passed directly from wildcard keyword args passed to
            the pipeline con

The `name` parameter is just used internally to distinguish working directories between pipeline nodes. There aren't any requirements on it except that needs to be unique amongst all pipelines that can be generated by the Analysis instance.

`citations` lists the publications that should be cited when using this pipeline. However, I plan to replace this bespoke solution with the third-party package [duecredit](https://pypi.org/project/duecredit/), which nipype uses.

The `name_maps` parameter should be passed directly from the keyword arguments of the pipeline constructor. It allows sub-classes to do funky things when manipulating methods defined in base classes such as mapping inputs and outputs onto different data-specs. However, going into the details of how it works is beyond the scope of this notebook.

### Adding nodes to a pipeline

The syntax for adding nodes to a pipeline is somewhat different to how it is done in Nipype. This is because it is modelled on the proposed syntax for [Nipype v2.0](https://github.com/nipy/nipype/projects/8), which aims to streamline code for workflow construction and make it easy to read.

Nodes are added to a pipeline using the `add` method

In [13]:
from arcana.pipeline import Pipeline
print(Pipeline.add.__doc__)


        Adds a processing Node to the pipeline

        Parameters
        ----------
        name : str
            Name for the node
        interface : nipype.Interface
            The interface to use for the node
        inputs : dict[str, (str, FileFormat) | (Node, str)]
            Connections from inputs of the pipeline and outputs of other nodes
            to inputs of node. The keys of the dictionary are the field names
            and the values are 2-tuple containing either the name of the data
            spec and the data format it is expected in for pipeline inputs or
            the sending Node and the the name of an output of the sending Node.
            Note that pipeline inputs can be specified outside this method
            using the 'connect_input' method and connections between nodes with
            the the 'connect' method.
        outputs : dict[str, (str, FileFormat)]
            Connections to outputs of the pipeline from fields of the
            interf

which in our BasicBrainAnalysis example looks like this

In [14]:
def smooth_mask_pipeline(self, **name_maps):

    pipeline = self.new_pipeline(
        'smooth_mask',
        desc="Smooths and masks a brain image",
        name_maps=name_maps,
        citations=[fsl_cite])

    # Smoothing process
    smooth = pipeline.add(
        'smooth',
        fsl.IsotropicSmooth(
            fwhm=self.parameter('smoothing_fwhm')),           # Param. passed from param-spec
        inputs={
            'in_file': ('magnitude', nifti_gz_format)},       # Input from data-spec
        outputs={
            'smooth': ('out_file', nifti_gz_format)},         # Output to data-spec
        requirements=[
            fsl_req.v('5.0.10')])                             # Requires FSL >= 5.0.10
    
    pipeline.add(
        'mask',
        fsl.ApplyMask(
            output_datatype=int),                             # Fixed param of pipeline
        inputs={
            'in_file': (smooth, 'out_file'),                  # Input from previous node
            'mask_file': ('brain_mask', nifti_gz_format)},    # Input from data-spec
        outputs={
            'smooth_masked': ('out_file', nifti_gz_format)},  # Output to data-spec
        requirements=[
            fsl_req.v('5.0.10')])                             # Requires FSL >= 5.0.10

    return pipeline


Key points to note are:

* All nodes need a unique name within the pipeline (as in Nipype)
* Stylistic convention dictates that constant interface traits are set when the interface is initialised
* `inputs` is a dictionary that maps the name of an input-trait of the interface to a 2-tuple consisting of either
  * a data-spec name (i.e. inputs of the pipeline) and the file-format/datatype the input is expected in (the format will be automatically converted if required)
  * an upstream node and name of the trait to connect from the upstream node
* `outputs` is a dictionary that maps data-spec names (i.e. outputs of the pipeline) to a 2-tuple consisting of the name of an output-trait and the file-format/datatype it is produced in.
* `requirements` are a list of `arcana.Requirement` objects that specify versions of external packages (e.g. FSL, SPM, MRtrix) that are required for the node to run.

### Merging and Splitting Pipelines Across Subjects/Visits

Arcana handles iteration over subjects and sessions in the background as implicitly specifed by the frequencies of inputs and outputs of the pipeline. However, in some cases you may need to join over all subjects/visits to create a summary statistic (e.g. mean), and then potentially use this variable back on an individual subject/visit level again (e.g. normalisation). As we saw in the in the morning "Advanced Nipype" section, these cases are handled by Nipype using iterators, map nodes and join nodes.

In Arcana join nodes are specified by providing the `joinsource` an `joinfield` parameters of when adding a node to a pipeline. These parameters work the same as they do for Nipype map nodes. Access to Arcana's implicit iterator nodes are exposed via the `self.SUBJECT_ID` and `self.VISIT_ID` variables. For example, in the `statistics_pipeline` of the `example.analysis.ToyAnalysis` we do a two-step merge, first over visits and then subjects to create the *per_dataset* metrics 'average' and 'std_dev' from the *per_session* 'selected_metric'.

In [15]:
def statistics_pipeline(self, **name_maps):
    pipeline = self.new_pipeline(
        name='statistics',
        name_maps=name_maps,
        desc="Calculate statistics")

    merge_visits = pipeline.add(
        'merge_visits',
        Merge(
            numinputs=1),
        inputs={
            'in1': ('selected_metric', text_format)},
        joinsource=self.VISIT_ID,
        joinfield=['in1'])

    merge_subjects = pipeline.add(
        'merge_subjects',
        Merge(
            numinputs=1,
            ravel_inputs=True),
        inputs={
            'in1': (merge_visits, 'out')},
        joinsource=self.SUBJECT_ID,
        joinfield=['in1'])

    concat = pipeline.add(
        'concat',
        ConcatFloats(),
        inputs={
            'in_files': (merge_subjects, 'out')})

    pipeline.add(
        'extract_metrics',
        ExtractMetrics(),
        inputs={
            'in_list': (concat, 'out_list')},
        outputs={
            'average': ('avg', float),
            'std_dev': ('std', float)})

    return pipeline

`self.SUBJECT_ID` and `self.VISIT_ID` can also be used to expand over all subject/visits again and to create a map node, simply provide the `iterfield` parameter when adding a node.

## Putting it all together

We can now put the three key components together (i.e. data & parameter specifications and pipeline constructor methods) to make a fully-functioning Analysis class.

In [16]:
class BasicBrainAnalysis(Analysis, metaclass=AnalysisMetaClass):
    """
    A baisc analysis class that demonstrates how Analysis classes work.
    """

    add_data_specs = [
        InputFilesetSpec('magnitude', nifti_gz_format,
                         desc="A magnitude image (e.g. T1w, T2w, etc..)"),
        OutputFilesetSpec('brain', nifti_gz_format,
                          'brain_extraction_pipeline',
                          desc="Skull-stripped magnitude image"),
        FilesetSpec('brain_mask', nifti_gz_format,
                    'brain_extraction_pipeline',
                    desc="Brain mask used for skull-stripping"),
        OutputFilesetSpec('smooth', nifti_gz_format, 'smooth_mask_pipeline',
                          desc="Smoothed magnitude image"),
        OutputFilesetSpec('smooth_masked', nifti_gz_format,
                          'smooth_mask_pipeline',
                          desc="Smoothed and masked magnitude image")]

    add_param_specs = [
        ParamSpec('smoothing_fwhm', 4.0,
                  desc=("The full-width-half-maxium radius of the smoothing "
                        "kernel"))]

    def brain_extraction_pipeline(self, **name_maps):

        pipeline = self.new_pipeline(
            'brain_extraction',
            desc="Extracts brain from full-head image",
            name_maps=name_maps,
            citations=[fsl_cite])

        pipeline.add(
            'bet',
            fsl.BET(
                mask=True),
            inputs={
                'in_file': ('magnitude', nifti_gz_format)},
            outputs={
                'brain': ('out_file', nifti_gz_format),
                'brain_mask': ('mask_file', nifti_gz_format)},
            requirements=[
                fsl_req.v('5.0.10')])

        return pipeline

    def smooth_mask_pipeline(self, **name_maps):

        pipeline = self.new_pipeline(
            'smooth_mask',
            desc="Smooths and masks a brain image",
            name_maps=name_maps,
            citations=[fsl_cite])

        # Smoothing process
        smooth = pipeline.add(
            'smooth',
            fsl.IsotropicSmooth(
                fwhm=self.parameter('smoothing_fwhm')),
            inputs={
                'in_file': ('magnitude', nifti_gz_format)},
            outputs={
                'smooth': ('out_file', nifti_gz_format)},
            requirements=[
                fsl_req.v('5.0.10')])

        pipeline.add(
            'mask',
            fsl.ApplyMask(),
            inputs={
                'in_file': (smooth, 'out_file'),
                'mask_file': ('brain_mask', nifti_gz_format)},
            outputs={
                'smooth_masked': ('out_file', nifti_gz_format)},
            requirements=[
                fsl_req.v('5.0.10')])

        return pipeline

## Methods to Create Publication Outputs

While not necessary, if you are creating a new Analysis class for your specific study, it is a nice idea to implement additional methods to generate all your publication outputs (figures, stats, etc...) within the Analysis class.

For example in the `BasicBrainAnalysis` class we have the method `plot_comparison`, which is implemented as follows

In [17]:
import matplotlib.pyplot as plt

class BasicBrainAnalysis(Analysis, metaclass=AnalysisMetaClass):
    """
    A baisc analysis class that demonstrates how Analysis classes work.
    """

    add_data_specs = [
        InputFilesetSpec('magnitude', nifti_gz_format,
                         desc="A magnitude image (e.g. T1w, T2w, etc..)"),
        OutputFilesetSpec('brain', nifti_gz_format,
                          'brain_extraction_pipeline',
                          desc="Skull-stripped magnitude image"),
        FilesetSpec('brain_mask', nifti_gz_format,
                    'brain_extraction_pipeline',
                    desc="Brain mask used for skull-stripping"),
        OutputFilesetSpec('smooth', nifti_gz_format, 'smooth_mask_pipeline',
                          desc="Smoothed magnitude image"),
        OutputFilesetSpec('smooth_masked', nifti_gz_format,
                          'smooth_mask_pipeline',
                          desc="Smoothed and masked magnitude image")]

    add_param_specs = [
        ParamSpec('smoothing_fwhm', 4.0,
                  desc=("The full-width-half-maxium radius of the smoothing "
                        "kernel"))]

    def brain_extraction_pipeline(self, **name_maps):

        pipeline = self.new_pipeline(
            'brain_extraction',
            desc="Extracts brain from full-head image",
            name_maps=name_maps,
            citations=[fsl_cite])

        pipeline.add(
            'bet',
            fsl.BET(
                mask=True),
            inputs={
                'in_file': ('magnitude', nifti_gz_format)},
            outputs={
                'brain': ('out_file', nifti_gz_format),
                'brain_mask': ('mask_file', nifti_gz_format)},
            requirements=[
                fsl_req.v('5.0.10')])

        return pipeline

    def smooth_mask_pipeline(self, **name_maps):

        pipeline = self.new_pipeline(
            'smooth_mask',
            desc="Smooths and masks a brain image",
            name_maps=name_maps,
            citations=[fsl_cite])

        # Smoothing process
        smooth = pipeline.add(
            'smooth',
            fsl.IsotropicSmooth(
                fwhm=self.parameter('smoothing_fwhm')),
            inputs={
                'in_file': ('magnitude', nifti_gz_format)},
            outputs={
                'smooth': ('out_file', nifti_gz_format)},
            requirements=[
                fsl_req.v('5.0.10')])

        pipeline.add(
            'mask',
            fsl.ApplyMask(),
            inputs={
                'in_file': (smooth, 'out_file'),
                'mask_file': ('brain_mask', nifti_gz_format)},
            outputs={
                'smooth_masked': ('out_file', nifti_gz_format)},
            requirements=[
                fsl_req.v('5.0.10')])

        return pipeline

    def plot_comparision(self, figsize=(12, 4)):

        for subj_i in self.subject_ids:
            for visit_i in self.visit_ids:
                f = plt.figure(figsize=figsize)
                f.suptitle('Subject "{}" - Visit "{}"'.format(subj_i, visit_i))
                for i, spec_name in enumerate(['magnitude', 'smooth',
                                               'brain_mask', 'smooth_masked']):
                    f.add_subplot(1, 4, i + 1)
                    self._plot_slice(spec_name, subj_i, visit_i)
                    plt.title(spec_name)
        plt.show()

    def _plot_slice(self, spec_name, subject_id=None, visit_id=None):
        # Load the image
        data = self.data(spec_name, derive=True).item(
            subject_id=subject_id, visit_id=visit_id).get_array()

        # Cut in the middle of the brain
        cut = int(data.shape[-1] / 2) + 10

        # Plot the data
        plt.imshow(np.rot90(data[..., cut]), cmap="gray")
        plt.gca().set_axis_off()

Notice how the `_plot_slice` method access the derived data using the `Analysis.data` method like we did in the "Applying Analysis Class" notebook. From the `FilesetSlice` it returns you can access a single data "item" using the `item` method. In Banana, the data array of `Fileset`s in standard image format can be accessed using the `get_arrray` method, which we then plot with Matplotlib.

## Exercise

Extend the `example.analaysis.BasicBrainAnalysis` class to add the `image_std` data-spec using the `nipype.interfaces.fsl.ImageStats` interface, which is the standard deviation of the smooth-masked image. Then run this analysis on the Tw-weighted images in the 'output/sample-datasets/depth1' dataset created in the "Applying Analysis Classes" notebook.

In [18]:
## Write your solution here

In [20]:
! fslstats -h

Usage: fslstats [preoptions] <input> [options]

preoption -t will give a separate output line for each 3D volume of a 4D timeseries
preoption -K < indexMask > will generate seperate n submasks from indexMask, for indexvalues 1..n where n is the maximum index value in indexMask, and generate statistics for each submask
Note - options are applied in order, e.g. -M -l 10 -M will report the non-zero mean, apply a threshold and then report the new nonzero mean

-l <lthresh> : set lower threshold
-u <uthresh> : set upper threshold
-r           : output <robust min intensity> <robust max intensity>
-R           : output <min intensity> <max intensity>
-e           : output mean entropy ; mean(-i*ln(i))
-E           : output mean entropy (of nonzero voxels)
-v           : output <voxels> <volume>
-V           : output <voxels> <volume> (for nonzero voxels)
-m           : output mean
-M           : output mean (for nonzero voxels)
-s           : output standard deviation
-S           : output s

In [35]:
from arcana import OutputFieldSpec
from example.analysis import BasicBrainAnalysis
from banana.requirement import fsl_req
from banana.citation import fsl_cite


class MyExtendedBasicBrainAnalysis(BasicBrainAnalysis, metaclass=AnalysisMetaClass):

    add_data_specs = [
        OutputFilesetSpec('smooth', mrtrix_image_format, 'smooth_mask_pipeline',
                          desc="Smoothed magnitude image in Mrtrix format"),
        OutputFieldSpec('image_std', float,
                        'image_std_pipeline',
                        desc="Standard deviation of the smoothed masked image")]
    
    def image_std_pipeline(self, **name_maps):
        
        pipeline = self.new_pipeline(
            'image_std_pipeline',
            desc="Calculates the standard deviation of the smooth masked image",
            name_maps=name_maps,
            citations=[fsl_cite])

        pipeline.add(
            'mask',
            fsl.ImageStats(
                op_string='-s'),
            inputs={
                'in_file': ('smooth_masked', nifti_gz_format)},
            outputs={
                'image_std': ('out_stat', nifti_gz_format)},
            requirements=[
                fsl_req.v('5.0.10')])

        return pipeline
        
print(MyExtendedBasicBrainAnalysis.static_menu(full=True))


__main__.MyExtendedBasicBrainAnalysis Menu 
-------------------------------------------

Inputs:
    magnitude : nifti_gz
        A magnitude image (e.g. T1w, T2w, etc..)

Intermediate:
    brain_mask : nifti_gz
        Brain mask used for skull-stripping

Outputs:
    brain : nifti_gz
        Skull-stripped magnitude image
    image_std : float
        Standard deviation of the smoothed masked image
    smooth : mrtrix_image
        Smoothed magnitude image in Mrtrix format
    smooth_masked : nifti_gz
        Smoothed and masked magnitude image

Parameters:
    smoothing_fwhm : float (4.0)
        The full-width-half-maxium radius of the smoothing kernel


In [36]:
from arcana import Dataset, FilesetFilter


my_analysis = MyExtendedBasicBrainAnalysis(
    'my_extended_analysis',  # The name needs to be the same as the previous version
    dataset=Dataset('output/sample-datasets/depth1', depth=1),
    processor='work',
    inputs=[
        FilesetFilter('magnitude', '.*T1w$', is_regex=True)])

my_analysis.derive('image_std')

191115-08:22:14,308 nipype.workflow INFO:
	 Workflow image_std_pipeline settings: ['check', 'execution', 'logging', 'monitoring']
191115-08:22:14,466 nipype.workflow INFO:
	 Running serially.
191115-08:22:14,468 nipype.workflow INFO:
	 [Node] Setting-up "image_std_pipeline.brain_extraction.brain_extraction_per_session_source" in "/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/work/image_std_pipeline/brain_extraction/_subject_id_sub1/_visit_id_VISIT/brain_extraction_per_session_source".
191115-08:22:14,476 nipype.workflow INFO:
	 [Node] Running "brain_extraction_per_session_source" ("arcana.repository.interfaces.RepositorySource")
191115-08:22:14,509 nipype.workflow INFO:
	 [Node] Finished "image_std_pipeline.brain_extraction.brain_extraction_per_session_source".
191115-08:22:14,510 nipype.workflow INFO:
	 [Node] Setting-up "image_std_pipeline.brain_extraction.brain_extraction_bet" in "/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-wor

191115-08:22:25,603 nipype.workflow INFO:
	 [Node] Running "brain_extraction_final" ("nipype.interfaces.utility.base.Merge")
191115-08:22:25,615 nipype.workflow INFO:
	 [Node] Finished "image_std_pipeline.brain_extraction.brain_extraction_final".
191115-08:22:25,618 nipype.workflow INFO:
	 [Node] Setting-up "image_std_pipeline.smooth_mask.smooth_mask_prereqs" in "/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/work/image_std_pipeline/smooth_mask/smooth_mask_prereqs".
191115-08:22:25,629 nipype.workflow INFO:
	 [Node] Running "smooth_mask_prereqs" ("nipype.interfaces.utility.base.Merge")
191115-08:22:25,639 nipype.workflow INFO:
	 [Node] Finished "image_std_pipeline.smooth_mask.smooth_mask_prereqs".
191115-08:22:25,641 nipype.workflow INFO:
	 [Node] Setting-up "image_std_pipeline.smooth_mask.smooth_mask_per_session_source" in "/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/work/image_std_pipelin

191115-08:22:37,569 nipype.workflow INFO:
	 [Node] Running "smooth_mask_mask" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/work/image_std_pipeline/smooth_mask/_subject_id_sub3/_visit_id_VISIT/smooth_mask_smooth/sub-03_ses-test_T1w_smooth.nii.gz -mas /Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/output/sample-datasets/depth1/sub3/my_extended_analysis/brain_mask.nii.gz /Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/work/image_std_pipeline/smooth_mask/_subject_id_sub3/_visit_id_VISIT/smooth_mask_mask/sub-03_ses-test_T1w_smooth_masked.nii.gz
191115-08:22:38,541 nipype.workflow INFO:
	 [Node] Finished "image_std_pipeline.smooth_mask.smooth_mask_mask".
191115-08:22:38,542 nipype.workflow INFO:
	 [Node] Setting-up "image_std_pipeline.smooth_mask.smooth_mask_per_s

191115-08:22:44,896 nipype.workflow INFO:
	 [Node] Setting-up "image_std_pipeline.image_std_pipeline.image_std_pipeline_per_session_source" in "/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/work/image_std_pipeline/image_std_pipeline/_subject_id_sub1/_visit_id_VISIT/image_std_pipeline_per_session_source".
191115-08:22:44,909 nipype.workflow INFO:
	 [Node] Running "image_std_pipeline_per_session_source" ("arcana.repository.interfaces.RepositorySource")
191115-08:22:44,923 nipype.workflow INFO:
	 [Node] Finished "image_std_pipeline.image_std_pipeline.image_std_pipeline_per_session_source".
191115-08:22:44,924 nipype.workflow INFO:
	 [Node] Setting-up "image_std_pipeline.image_std_pipeline.image_std_pipeline_mask" in "/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/work/image_std_pipeline/image_std_pipeline/_subject_id_sub1/_visit_id_VISIT/image_std_pipeline_mask".
191115-08:22:44,929 nipype.workf

191115-08:22:46,142 nipype.workflow INFO:
	 [Node] Finished "image_std_pipeline.image_std_pipeline.image_std_pipeline_final".


TypeError: 'float' object is not callable

In [39]:
for std in my_analysis.data('image_std'):
    print('Subject/visit ({}/{}): {} '.format(std.subject_id, std.visit_id, std.value))

Subject/visit (sub1/VISIT): 126.144342 
Subject/visit (sub2/VISIT): 140.465662 
Subject/visit (sub3/VISIT): 152.161282 
